<a href="https://colab.research.google.com/github/datafriends/Workshops/blob/master/How_to_hack_a_hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to hack a Hack with R

## 1. Specify the problem

Context: 
- In Australia a Credit License enables you to credit to others. 
- A Financial Services license is about providing guidance to people buying financial products. You can't engage in credit activities with a FSL.


Prompt: 
- Has the regulatory scrutiny in Australia recently led to a change in distribution of
credit licenses vs financial services licenses? 
- Is there evidence that greater scruitany in banking is resulting in fewer Credit Licenses and more Financial Services Licenses?

## 2. Investigate the dataset

In [0]:
# load some libraries we will need
library(tidyverse)
library(lubridate)


options(repr.plot.width=6, repr.plot.height=4)

In [0]:
install.packages("shiny")

In [0]:
library(shiny)

In [0]:
# download datasets from data.gov.au

# FSL dataset
afs_file <- "https://data.gov.au/data/dataset/ab7eddce-84df-4098-bc8f-500d0d9776d1/resource/1fc2b334-74ae-4588-85aa-83e3a8c9f2cc/download/afs_lic_201906.tsv"

In [0]:
afs <- read.table(afs_file, sep = '\t', header = TRUE, comment.char="",  quote ="\"")



In [0]:
# credit dataset
credit_file <- "https://data.gov.au/data/dataset/fa0b0d71-b8b8-4af8-bc59-0b000ce0d5e4/resource/3abf1383-c4e5-4c1a-8331-8434b17b6f10/download/credit_lic_201906.tsv"


In [0]:
# YOUR TURN: read in the credit license file: call it "credit" in this cell.

In [0]:
tail(afs, 10)

In [0]:
# MY SOLUTION: Ok, this was an easy one!
credit <- read.table(credit_file, sep = '\t', header = TRUE, comment.char="",  quote ="\"")

In [0]:
tail(credit, 10)

We can find out some basic information about our datasets. I'll do the afs data, you guys jump in and tell me about the credit data.

In [0]:
str(afs)

In [0]:
summary(afs)

In [0]:
# YOUR TURN: credit dataset investigation:

# 1. investigate the credit dataset.

# 2. How do we change the names of the credit and the afs datasets so they have common names? 

## 3. Data Wrangling

We need to drop the AFS_ prefix from the afs dataset and the CRED_prefix from the credit dataset variables.

In [0]:
head(afs,1)

In [0]:
head(credit,1)

In [0]:
names(afs)
names(credit)

In [0]:
# I'll clean the afs dataset, below you can clean the credit dataset

rename_cols <- function(prefix, dataset) {
  gsub(prefix, "", names(dataset))
}


In [0]:
# remove the AFS_ prefix
names(afs) <- rename_cols("AFS_", afs)

# oops there is also a random ADD_ which needs to be removed in names like AFS_LIC_ADD_LOCAL
names(afs) <- rename_cols("ADD_", afs)
names(afs)[names(afs) == "LIC_LOCAL"] <-"LIC_LOCALITY"

In [0]:
# YOUR TURN: drop the _CRED prefix from the credit file

names(credit)

In [0]:
names(afs)

In [0]:
# MY SOLUTION: drop the _CRED prefix from the credit file

names(credit) <- rename_cols("CRED_", credit)

In [0]:
names(credit)

In [0]:
# keep the common variables

common_names <- names(afs)[names(afs) %in% names(credit)]

afs <- select(afs, common_names)


In [0]:
# YOUR TURN: select the common_names in the credit dataset

In [0]:
names(afs)

In [0]:
names(credit)

In [0]:
# MY SOLUTION: select the common_names in the credit dataset
credit <- select(credit, common_names)

In [0]:
# create a combined dataset

dat <- bind_rows(afs, credit)

str(dat)

In [0]:
# we need to change the Start_Date to date format

dat <- mutate(dat, LIC_START_DT = dmy(LIC_START_DT))

str(dat)

## 4. Exploratory Data Analysis

In [0]:
summary_data <- select(dat, REGISTER_NAME, LIC_NUM, LIC_START_DT) %>% # variables we wish to keep
                    mutate(year = year(LIC_START_DT), # the year part of LIC_START_DATE
                           mon = sprintf("%02d", month(LIC_START_DT))) %>% # the month part of LIC_START_DATE
                    group_by(year, mon, REGISTER_NAME) %>% # roll up the data by year and month and license type
                    summarise(cnt = n()) %>% # count the rows
                    filter(year >= 2000) # subset for the data after 2000

In [0]:
summary_data

In [0]:
# YOUR TURN: run the following code, what is the problem with the data before year 2000?
temp_data <- select(dat, REGISTER_NAME, LIC_NUM, LIC_START_DT) %>% # variables we wish to keep
                    mutate(year = year(LIC_START_DT)) %>% # the year part of LIC_START_DATE
                    group_by(year, REGISTER_NAME) %>% # roll up the data by year and month and license type
                    summarise(cnt = n())

temp_data

In [0]:
# we can plot the data like this, what do you notice in this plot?:

ggplot(data = summary_data, aes(year, cnt)) + # pass in the dataset and the variables year and count
    geom_bar(stat = "identity", aes(fill = REGISTER_NAME)) + # do a bar plot, but color by license type
    facet_wrap(REGISTER_NAME ~ .) # facet the data to produce 2 graphs for each license type.


In [0]:
# YOUR TURN: can you create the summary data again, but this time filter for year >= 2012

# extra marks if you can reproduce this plot above but this time facet by state LIC_STATE?

table(summary_data$year)

table(temp_data$year)

In [0]:
# MY SOLUTION

summary_data <- select(dat, REGISTER_NAME, LIC_NUM, LIC_STATE, LIC_START_DT) %>%
    mutate(year = year(LIC_START_DT),
           mon = sprintf("%02d", month(LIC_START_DT))) %>%
    group_by(year, mon, REGISTER_NAME, LIC_STATE) %>%
    summarise(cnt = n()) %>%
    filter(year >= 2012) # this is the line that's different

ggplot(data = summary_data, aes(year, cnt)) +
    geom_bar(stat = "identity", aes(fill = REGISTER_NAME)) +
    facet_wrap(LIC_STATE ~ .)

## 5. Predictive model to explain what we are seeing

So, the regulations really kicked in from about 2016 and we can see more AFS than Credit Licenses then. So pretending we didn't have data from 2016-2018 what would we have expected from the 2012-2016 data? 

To answer this question let's build a simple model.

In [0]:
summary_data <- select(dat, REGISTER_NAME, LIC_NUM, LIC_STATE, LIC_START_DT) %>%
    mutate(year = year(LIC_START_DT),
           mon = sprintf("%02d", month(LIC_START_DT))) %>%
    group_by(year, mon, REGISTER_NAME, LIC_STATE) %>%
    summarise(cnt = n()) %>%
    filter(year >= 2012)

In [0]:
# our training data is 2012-2016 and we wish to predict data from 2016-2018

train <- filter(summary_data, year < 2016 & year >= 2012)
test <- filter(summary_data, year >= 2016 & year <= 2018)

In [0]:
table(train$year)
table(test$year)

In [0]:
# here's what a histogram of the count variable looks like

hist(train$cnt)

In [0]:
# here's the logarithm of this variable
hist(log(train$cnt))

In [0]:
# does the logarithm of the count variable follow a normal distribution?
qqnorm(log(train$cnt))
qqline(log(train$cnt))

In [0]:
# normal linear regression of LIC_STATE, REGISTER_NAME and mon to predict logarithm of count

mod <- glm(data=train, log(cnt) ~ LIC_STATE + REGISTER_NAME + mon)
summary(mod)

In [0]:
# YOUR TURN: drop mon as an explanatory variable 

head(train)

In [0]:
# MY SOLUTION

mod <- glm(data=train, log(cnt) ~ LIC_STATE + REGISTER_NAME)
summary(mod)

In [0]:
# now we predict our model from the training dataset onto the test dataset

test$pred <- predict(mod, newdata=test)
test$pred <- exp(test$pred)

In [0]:
head(test)

In [0]:
# now we wish to change the structure of our test data from wide to long to make it easier to plot
test_dat <- gather(test, variable, value, cnt:pred)

In [0]:
head(test_dat)
tail(test_dat)

## 6. Data product - web app actual vs expected in 2016 using data to 2015

In [0]:
ui <- fluidPage(
    titlePanel("Predicted 2016-2018 using data from 2012-2015"),
    sidebarLayout(
        sidebarPanel(
            selectInput(inputId = "subset_state", label = "Pick a state: ", 
                               choices = unique(test_dat$LIC_STATE))),
        mainPanel(plotOutput(outputId = "myfigure"))
    )
)


server <- function(input, output) {
    create_subset <- reactive(test_dat %>%
                                  filter(LIC_STATE == input$subset_state))
    
    output$myfigure <- renderPlot(
                            ggplot(create_subset()) +
                                      geom_bar(aes(x = year, y = value, fill=variable), stat="identity") + 
                                        facet_wrap(REGISTER_NAME ~ variable) +
                                        theme(axis.text.x = element_text(angle = 90, hjust = 1))
                                      
                            )
}


In [0]:
# Run app: this unfortunately won't work from Colab, but will run in RStudio :( 
# shinyApp(ui,server)

In [0]:
ggplot(test_dat[test_dat$LIC_STATE == 'NSW',]) + 
geom_bar(aes(x = year, y = value, fill=REGISTER_NAME), stat="identity") + 
                                        facet_wrap(REGISTER_NAME ~ variable) +
                                        theme(axis.text.x = element_text(angle = 90, hjust = 1))